In [35]:
# Import necessary modules
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import tslearn
from tslearn.datasets import UCR_UEA_datasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler, TimeSeriesScalerMinMax

In [36]:
dataset_name = 'ElectricDevices'

In [37]:
# Load and preprocess the time series data
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)

In [38]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((8926, 96, 1), (8926,), (7711, 96, 1), (7711,))

In [39]:
#normalize the data
X_train = TimeSeriesScalerMinMax().fit_transform(X_train)
X_test = TimeSeriesScalerMinMax().fit_transform(X_test)

In [40]:
# Resample the data to 128 time steps
X_train = TimeSeriesResampler(sz=128).fit_transform(X_train)
X_test = TimeSeriesResampler(sz=128).fit_transform(X_test)

In [41]:
# Convert the data to torch tensors
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

In [42]:
#Datasets
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
#Dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=time_series.shape[1], out_channels=8, kernel_size=3, stride=1)
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(in_features=16, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [5]:
# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=time_series.shape[1], hidden_size=8, num_layers=2)
        self.lstm2 = nn.LSTM(input_size=8, hidden_size=16, num_layers=2)
        self.fc1 = nn.Linear(in_features=16, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = x[-1, :, :]
        x = self.fc1(x)
        x = self.fc2(x)
        return x




In [6]:
# Define the combined CNN-LSTM model
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        self.cnn = CNN()
        self.lstm = LSTM()

    def forward(self, x):
        x_cnn = self.cnn(x)
        x_lstm = self.lstm(x)
        x = torch.cat((x_cnn, x_lstm), dim=1)
        return x